In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import itertools as it
import scipy.stats.qmc as ssq
from scipy.stats import chi2, truncnorm
from seaborn import displot
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space.space import Real, Integer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, KBinsDiscretizer
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer

np.set_printoptions(threshold=100_000)
X = pd.read_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\ensemble_output\results\default\parameters.csv")
# X = X[['gamma_g', 't_peak', 'gamma_d', 'sd_antarctic',
#        'rho_greenland', 'rho_gmsl', 'temperature_0', 'ocean_heat_0', 'Q10',
#        'CO2_diffusivity', 'heat_diffusivity', 'rf_scale_aerosol',
#        'climate_sensitivity', 'thermal_alpha', 'greenland_a', 'greenland_b',
#        'greenland_alpha', 'greenland_beta', 'anto_alpha', 'antarctic_mu',
#        'antarctic_precip0', 'antarctic_runoff_height0', 'antarctic_lambda',
#        'antarctic_temp_threshold', 'lw_random_sample']].copy()
Y = pd.read_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\ensemble_output\results\default\gmslr.csv")
Y = Y.mean(axis=1).rename('output')
# X.t_peak = X.t_peak.astype('category')
obj = X.join(Y, how='left')
# obj.sort_values(by='t_peak',ascending=True, inplace=True)
# Rescaling the dataset
# min_max = MinMaxScaler()
# min_max.fit(obj.iloc[:,:-1],obj.iloc[:,-1])

# obj_scaled = min_max.transform(obj.iloc[:,:-1])
# obj_y = obj.iloc[:,-1].copy()


In [2]:
nun = len(X.t_peak.unique())
discret = KBinsDiscretizer(n_bins=nun, encode='ordinal', random_state=0)
Y_binned = discret.fit_transform(Y.to_numpy().reshape(-1,1)).ravel()
smote = SMOTENC(categorical_features=['t_peak'],sampling_strategy='auto', random_state=0)
X_res , y_res = smote.fit_resample(X, Y_binned)

In [3]:
Y1 = pd.Series(discret.inverse_transform(y_res.reshape(-1,1)).ravel(), name='output')
obj1 = X_res.join(Y1, how='left')

In [4]:
obj1.sort_values(by='t_peak', inplace=True)

In [55]:
# Redefine model
val = train_test_split(obj3060.iloc[:,:-1],obj3060.iloc[:,-1], train_size=0.8, random_state=0)
# val = TimeSeriesSplit(n_splits=4, max_train_size=60_066)
#Darnell MSE8 n_estimators=199, max_depth=26, max_features=30, min_samples_leaf=2,n_jobs=10,
# Darnell MSE10:max_depth=36, max_features=33, n_estimators=300,
# max_depth=38, max_features=17,n_estimators=300
#==================================================================================================
RF = RandomForestRegressor(n_estimators=295, max_features=47, max_depth=55, n_jobs=20,random_state=0)
RF.fit(val[0].to_numpy(), val[2].to_numpy())

RandomForestRegressor(max_depth=55, max_features=47, n_estimators=295,
                      n_jobs=20, random_state=0)

In [56]:
RF.feature_importances_

array([0.0076261 , 0.00789796, 0.00783383, 0.00856189, 0.00934584,
       0.00818608, 0.00783684, 0.00802969, 0.00736885, 0.00975159,
       0.0076832 , 0.00810338, 0.00798281, 0.00829745, 0.00827155,
       0.0081994 , 0.00784355, 0.00797682, 0.0079825 , 0.00753266,
       0.00786261, 0.01665105, 0.00772726, 0.00857783, 0.00745177,
       0.0236806 , 0.29376464, 0.00977019, 0.01184858, 0.01363464,
       0.00963379, 0.01521749, 0.00666717, 0.00868914, 0.00872477,
       0.0074555 , 0.00784829, 0.00773902, 0.01599659, 0.00783756,
       0.0091054 , 0.00804184, 0.00851691, 0.0106149 , 0.007976  ,
       0.00799718, 0.00715169, 0.13858481, 0.14292079])

In [6]:
val = train_test_split(obj1.iloc[:,:-1],obj1.iloc[:,-1],shuffle=False, train_size=0.8, random_state=0)
# val = TimeSeriesSplit(n_splits=4, max_train_size=60_066)
#Darnell MSE8 n_estimators=199, max_depth=26, max_features=30, min_samples_leaf=2,n_jobs=10,
# Darnell MSE10:max_depth=36, max_features=33, n_estimators=300,
# max_depth=38, max_features=17,n_estimators=300
#==================================================================================================
RF = RandomForestRegressor(n_estimators=295, max_features=47, max_depth=55, n_jobs=20,random_state=0)

In [ ]:
RF.fit(val[0], val[2])

In [ ]:
print(f'Train MSE={np.mean(np.square(val[2].to_numpy()-RF.predict(val[0].to_numpy())))}')
print(f'Test MSE={np.mean(np.square(val[3].to_numpy()-RF.predict(val[1].to_numpy())))}')
# print(f'Importances:\n{RF.feature_importances_}')

In [9]:
parameter_space = dict()
parameter_space['n_estimators'] = Integer(30,300)
# parameter_space['min_samples_split'] = Integer(2,5, prior='uniform')
# parameter_space['min_samples_leaf'] = Integer(1,5)
parameter_space['max_features'] = Integer(10,53)
parameter_space['max_depth'] = Integer(10,80)
# parameter_space['ccp_alpha'] = Real(1e-4, 5e-4)

In [ ]:
bae = BayesSearchCV(estimator=RF,search_spaces=parameter_space,verbose=2, n_iter=100, scoring='neg_mean_squared_error', n_points=6, cv=val, n_jobs=24, random_state=0)
bae.fit(obj1.iloc[:,:-1], obj1.iloc[:,-1])
res = pd.DataFrame(bae.cv_results_)
res.to_excel(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Hyperparameter_Tuning\Darnell_HT_MSE_12.xlsx")
res.to_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Hyperparameter_Tuning\Darnell_HT_MSE_12.csv")
print(f'Best Parameters:{bae.best_params_} with MSE: {bae.best_score_}')

In [24]:
rng = np.random.default_rng(seed=0)
def bootstrapp(y_true,y_pred,n_boots):
    rows = len(y_true)
    error = (y_true - y_pred).to_numpy()
    S_CI = np.zeros((n_boots,))
    for i in range(n_boots):
        indx = rng.choice(rows, size=rows, replace=True)
        S_CI[i] = np.mean(np.square(error[indx]))
    return S_CI

In [25]:
def pvi1(data: pd.DataFrame, y_true: np.ndarray[float], f: callable,*, n_boots: int, alpha: float, column_set: dict) -> tuple[pd.DataFrame]:
    rows, columns = data.shape
    npv_columns = column_set
    V_y = np.var(y_true, ddof=1)
    u = np.zeros((columns,))
    CI = dict()
    # rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    rolled_data = np.roll(data.copy(),rows//2, axis=0)
    # for cat,col in npv_columns.items():
    for col in npv_columns:
        d = rolled_data.copy()
        d[:, col] = data[:,col]
        y_pred = f(d)
        u[col] = 1-np.mean(np.square(y_true - y_pred))/(2*V_y)
    #     S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
    #     p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
    #     CI[col] = [p0,p1,p1-p0]
    # CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    # CI.index.name = 'Interactions'
    # CI.columns.name = 'Confidence Interval Difference'
    # SI = pd.DataFrame(u.values(), index=u.keys(), columns=['$P_{i}$'])
    return u

In [26]:
def mod_pvi1(data: pd.DataFrame, y_true: np.ndarray[float], f: callable,*, n_boots: int, alpha: float, column_set: dict) -> tuple[pd.DataFrame]:
    rows, columns = data.shape
    npv_columns = column_set
    V_y = np.var(y_true, ddof=1)
    u = dict()
    CI = dict()
    rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    # rolled_data = np.roll(data.copy(), rows//2, axis=0)
    for cat, col in npv_columns.items():
        d = rolled_data.copy()
        d[col] = data[col].to_numpy()
        y_pred = f(d)
        u[cat] = 1-np.mean(np.square(y_true - y_pred))/(2*V_y)
        S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
        p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
        CI[cat] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'
    SI = pd.DataFrame(u.values(), index=u.keys(), columns=['$P_{i}$'])
    return u,CI

In [27]:
def pvi_t(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,n_boots: int, alpha: float, column_set: list[str]) -> np.ndarray[float]:
    '''
    ===Total-Order Permutation Variable Importances===

        data: your training dataset
        y_pred: your model output (specifically the prediction of your 'data' arg)
        f: in this case, simply RF.predict
    '''
    rows, columns = data.shape
    npv_columns = column_set
    u = np.zeros((columns,))
    V_y = np.var(y_true, ddof=1)
    CI = dict()
    for col in npv_columns:
        d = data.copy()
        d[:, col] = np.roll(data[:, col],rows//2, axis=0)
        y_pred = f(d)
        u[col] = np.mean(np.square(y_true - y_pred))/(2*V_y)
    #     S_CI = bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
    #     p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
    #     CI[col] = [p0,p1,p1-p0]
    # CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    # CI.index.name = 'Interactions'
    # CI.columns.name = 'Confidence Interval Difference'
    # S_t = pd.DataFrame(u.values(), index=u.keys(), columns=[r'$P_{\tau}$'])
    return u

In [28]:
def mod_pvi_t(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,n_boots: int, alpha: float, column_set: list[str]) -> np.ndarray[float]:
    '''
    ===Total-Order Permutation Variable Importances===

        data: your training dataset
        y_pred: your model output (specifically the prediction of your 'data' arg)
        f: in this case, simply RF.predict
    '''
    rows, columns = data.shape
    npv_columns = column_set
    u = dict()
    V_y = np.var(y_true,ddof=1 )
    CI = dict()
    for cat,col in npv_columns.items():
        d = data.copy()
        d[col] = np.roll(data[col],rows//2, axis=0)
        y_pred = f(d)
        u[cat] = np.mean(np.square(y_true - y_pred))/(2*V_y)
        S_CI = bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
        p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
        CI[cat] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'
    S_t = pd.DataFrame(u.values(), index=u.keys(), columns=[r'$P_{\tau}$'])
    return u, CI

In [29]:
def pvi2(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,S_I: np.ndarray[float] ,n_boots: int, alpha: float, columns_set: dict) -> tuple[np.ndarray[float], list]:
    '''
    ===Second-Order Permutation Variable Importances (Full Matrix)===

        data: your training dataset
        y_pred: your model output (specifically the y_true value)
        f: in this case, simply RF.predict

        S_I: First-Order Importances
        n_boots: number of bootstrap samples
        alpha: parameter for confidence interval [alpha, 1-alpha]
        S_CI: sampling distribution 
        CI: Confidence Intervals
    '''
    rows, columns = data.shape
    npv_columns = columns_set
    n = n_boots
    S = np.zeros((columns, columns))
    CI = dict()
    V_y = np.var(y_true, ddof=1)
    # rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    rolled_data = np.roll(data.copy(),rows//2, axis=0)
    for col1,col2 in it.combinations(npv_columns, r=2):
        d = rolled_data.copy()
        d[:,[col1, col2]] = data[:,[col1,col2]]
        y_pred = f(d)
        S_cat1 = S_I[col1]
        S_cat2 = S_I[col2]
        print(f'First-Order:\t{S_cat1, S_cat2}')
        # print(f'First Part:\t{np.mean(np.square(y_true - f(d)))/(2*V_y)}')
        # print(f'Second Part:\t{S_cat1}')
        # print(f'Third Part:\t{S_cat2}')
        S[col1, col2] = 1 - np.mean(np.square(y_true - y_pred))/(2*V_y) - S_cat1 - S_cat2
        print(f'Second-Order:\t{S[col1,col2]}')

    #     S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y) - S_cat1 - S_cat2
    #     p0,p1= np.quantile(S_CI, [alpha,1-alpha])
    #     CI[(col1, col2)] = [p0,p1,p1-p0]
    # CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    # CI.index.name = 'Interactions'
    # CI.columns.name = 'Confidence Interval Difference'

    # S = pd.DataFrame(S.values(), index=S.keys(), columns=[r'$P_{ik}$'])
    return S

In [30]:
def mod_pvi2(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,S_I: np.ndarray[float] ,n_boots: int, alpha: float, columns_set: dict) -> tuple[np.ndarray[float], list]:
    '''
    ===Second-Order Permutation Variable Importances (Full Matrix)===

        data: your training dataset
        y_pred: your model output (specifically the y_true value)
        f: in this case, simply RF.predict

        S_I: First-Order Importances
        n_boots: number of bootstrap samples
        alpha: parameter for confidence interval [alpha, 1-alpha]
        S_CI: sampling distribution 
        CI: Confidence Intervals
    '''
    rows, columns = data.shape
    npv_columns = columns_set
    n = n_boots
    S = dict()
    CI = dict()
    V_y = np.var(y_true, ddof=1)
    rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    # rolled_data = np.roll(data.copy(), rows//2, axis=0)
    for (cat1,col1),(cat2, col2) in it.combinations(npv_columns.items(), r=2):
        col = col1+col2
        d = rolled_data.copy()
        d[col] = data[col].to_numpy()
        y_pred = f(d)
        S_cat1 = S_I[cat1]
        S_cat2 = S_I[cat2]
        S[(cat1, cat2)] = 1 - np.mean(np.square(y_true - y_pred))/(2*V_y) - S_cat1 - S_cat2
        S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y) - S_cat1 - S_cat2 # Bootstrap Process
        p0,p1= np.quantile(S_CI, [alpha,1-alpha])
        CI[(cat1, cat2)] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'

    # S = pd.DataFrame(S.values(), index=S.keys(), columns=[r'$P_{ik}$'])
    return S, CI

In [31]:
Variables = ['sd_temp', 'sd_ocean_heat', 'sd_glaciers', 'sd_greenland','sd_antarctic',
              'sd_gmsl', 'rho_temperature','rho_ocean_heat', 
              'rho_glaciers', 'rho_greenland', 'rho_antarctic','rho_gmsl']
Climate = ['CO2_0', 'N2O_0', 'temperature_0','ocean_heat_0','Q10', 'CO2_fertilization',
           'CO2_diffusivity', 'heat_diffusivity', 'rf_scale_aerosol','climate_sensitivity']
Glaciers = ['glaciers_v0','glaciers_s0','glaciers_beta0', 'glaciers_n']
Thermals = ['thermal_s0', 'thermal_alpha']

Greenland = ['greenland_a', 'greenland_b','greenland_alpha', 'greenland_beta']
Antarctic = [ 'antarctic_gamma', 'antarctic_alpha','antarctic_mu', 'antarctic_nu',
              'antarctic_precip0', 'antarctic_kappa','antarctic_flow0', 'antarctic_runoff_height0',
                'antarctic_c','antarctic_bed_height0', 'antarctic_slope', 'antarctic_lambda','antarctic_temp_threshold',
                'antarctic_s0','anto_alpha', 'anto_beta']

In [32]:
# column_set = val[0].columns
# print(column_set)
npv_column_set = dict()
npv_column_set['Variables'] = Variables
npv_column_set['Climate'] = Climate
# npv_column_set['Glaciers'] = Glaciers
npv_column_set['Thermals'] = Thermals
npv_column_set['Greenland'] = Greenland
npv_column_set['Antarctic'] = Antarctic
npv_column_set['Emissions'] = ['gamma_g', 't_peak', 'gamma_d', 'lw_random_sample']

In [57]:
PI= pvi1(val[0].to_numpy(), val[2].to_numpy(), RF.predict, n_boots=1000, alpha=0.05, column_set=range(len(val[0].columns)))
PIk, CIk= pvi2(val[0].to_numpy(), val[2].to_numpy(), RF.predict,S_I=PI, n_boots=1000, alpha=0.05, columns_set=range(len(val[0].columns)))
print(PI)
print(PIk)

First-Order:	(0.30283277896206695, 0.3031694628315802)
Second-Order:	-0.29885630487558346
First-Order:	(0.30283277896206695, 0.3030293066041312)
Second-Order:	-0.2988369776497797
First-Order:	(0.30283277896206695, 0.3046638327562353)
Second-Order:	-0.2989073211289537
First-Order:	(0.30283277896206695, 0.3047757892636195)
Second-Order:	-0.2989403280063302
First-Order:	(0.30283277896206695, 0.3036274909992027)
Second-Order:	-0.29886141773328245
First-Order:	(0.30283277896206695, 0.3032741055314556)
Second-Order:	-0.2988548233471413
First-Order:	(0.30283277896206695, 0.3030375106314571)
Second-Order:	-0.2988445294244241
First-Order:	(0.30283277896206695, 0.3028086166954488)
Second-Order:	-0.29883297162858447
First-Order:	(0.30283277896206695, 0.3064823943427514)
Second-Order:	-0.29897624816334545
First-Order:	(0.30283277896206695, 0.30294877192612113)
Second-Order:	-0.2988372549691566
First-Order:	(0.30283277896206695, 0.30399132314041644)
Second-Order:	-0.29889528556481415
First-Order:	(

KeyboardInterrupt: 

In [25]:
def donut_chart(values: np.ndarray, values1: np.ndarray, labels: list,labels1: list, props: dict, kw: dict):
    fig, ax= plt.subplots(1,2,dpi=500, figsize=(20,25))
    wedges0, text0 = ax[0].pie(values,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
    for i, p in enumerate(wedges0):
        ang = (p.theta2-p.theta1)/2 + p.theta1
        x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
        horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
        connectionstyle = f'angle, angleA=0, angleB={ang}'
        kw['arrowprops'].update({'connectionstyle':connectionstyle})
        ax[0].annotate(f'{values[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
    ax[0].set_title('SNEASY-BRICK: First & Second-Order PVi')
    ax[0].legend(labels=labels,loc='center', title='PVi Indices:' )

    wedges1, text1 = ax[1].pie(values1,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
    for i, p in enumerate(wedges1):
        ang = (p.theta2-p.theta1)/2 + p.theta1
        x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
        horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
        connectionstyle = f'angle, angleA=0, angleB={ang}'
        kw['arrowprops'].update({'connectionstyle':connectionstyle})
        ax[1].annotate(f'{values1[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
    ax[1].set_title('SNEASY-BRICK: Total-Order PVi')
    ax[1].legend(labels=labels1,loc='center', title='PVi Indices:' )
    plt.show()

In [21]:
PT= mod_pvi_t(val[0], val[2], RF.predict, n_boots=1000, alpha=0.05, column_set=npv_column_set)
PT

({'Variables': 0.015095430276762934,
  'Climate': 0.32375627783499483,
  'Thermals': 0.008758090734658521,
  'Greenland': 0.014086980856964519,
  'Antarctic': 0.2526646463020993,
  'Emissions': 0.5727934689329497},
 Confidence Interval Difference       5th      95th  Quantile Difference
 Interactions                                                           
 Variables                       0.014623  0.015590             0.000967
 Climate                         0.315944  0.332229             0.016285
 Thermals                        0.008467  0.009045             0.000578
 Greenland                       0.013752  0.014454             0.000702
 Antarctic                       0.245806  0.259209             0.013403
 Emissions                       0.562302  0.583772             0.021471)

In [19]:
PIK,CIK= mod_pvi2(val[0], val[2], RF.predict,S_I=PI, n_boots=1000, alpha=0.05, columns_set=npv_column_set)
PIK = pd.DataFrame(PIK.values(), index=PIK.keys(), columns=[r'$P_{ik}$'])

In [20]:
PIK

$P_{ik}$
Variables Climate   -0.051538
          Thermals  -0.035164
          Greenland -0.037669
          Antarctic -0.052738
          Emissions -0.050539
Climate   Thermals  -0.035737
          Greenland -0.041018
          Antarctic -0.055184
          Emissions -0.050734
Thermals  Greenland -0.034807
          Antarctic -0.037930
          Emissions -0.037399
Greenland Antarctic -0.044441
          Emissions -0.043071
Antarctic Emissions -0.030327

In [ ]:
pd.DataFrame(PI.values(), index=PI.keys()).sum(axis=0)

In [42]:
props = dict()
kw = dict()
props['textprops'] = {'fontsize':14}
props['startangle'] = -270
props['wedgeprops'] = {'width': 0.2}
kw['arrowprops'] = dict(arrowstyle='-')
kw['zorder'] = 0
kw['va'] = 'center'
colors = ['#CF0057','#FF0054','#F72585', '#B5179E', '#7209B7', '#560BAD','#8F3DC1','#A96FD1',
          '#C3A3E0', '#8F6FD1', '#480CA8', '#3A0CA3','#141440','#A3A3E0', '#6F6FD1', '#3D3DC1',
          '#4361EE','#4CC9F0','#1DE48D','#6B007B','#B51CE4','#E044A7', '#744EC2']
props['colors'] = colors

In [33]:
PI = pd.DataFrame(PI.values(), index=PI.keys(), columns=[r'$P_{i}$'])

In [34]:
inner = PI.to_dict()[r'$P_{i}$']
inner = {key: value for key, value in inner.items() if value>0}

In [35]:
left_inner = PIK.to_dict()[r'$P_{ik}$']
left_inner = {','.join(key): value for key,value in left_inner.items()}

In [ ]:
inner

In [37]:
merged_indices = inner|left_inner
labels = list(merged_indices.keys())
values = list(merged_indices.values())

In [38]:
PT, CT = PT

In [39]:
labels1 = list(PT.keys())
values1 = list(PT.values())

In [ ]:
merged_indices

In [ ]:
donut_chart(values, values1, labels, labels1, props, kw)

In [ ]:
fig, ax= plt.subplots(1,1,dpi=500, figsize=(8,8))
wedges0, text0 = ax.pie(values,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
for i, p in enumerate(wedges0):
    ang = (p.theta2-p.theta1)/2 + p.theta1
    x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
    horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
    connectionstyle = f'angle, angleA=0, angleB={ang}'
    kw['arrowprops'].update({'connectionstyle':connectionstyle})
    ax.annotate(f'{values[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
ax.set_title('SNEASY-BRICK: First & Second-Order PVi')
ax.legend(labels=labels,loc='center', title='PVi Indices:' , prop={'size':6})
plt.show()

In [ ]:
fig, ax= plt.subplots(1,1,dpi=250, figsize=(8,8))
wedges0, text0 = ax.pie(values1,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
for i, p in enumerate(wedges0):
    ang = (p.theta2-p.theta1)/2 + p.theta1
    x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
    horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
    connectionstyle = f'angle, angleA=0, angleB={ang}'
    kw['arrowprops'].update({'connectionstyle':connectionstyle})
    ax.annotate(f'{values1[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
ax.set_title('SNEASY-BRICK: Total-Order PVi')
ax.legend(labels=labels1,loc='center', title='PVi Indices:' , prop={'size':6})
plt.show()

In [ ]:
PI_CI = PI.join(CI, how='left')
print(PI_CI.to_latex(float_format='{:.4f}'.format))

In [47]:
PT = pd.DataFrame(PT.values(), index=PT.keys(), columns=[r'$P_{\tau}$'])

In [ ]:
PT_CT = PT.join(CT, how='left')
print(PT_CT.to_latex(float_format='{:.4f}'.format))

In [ ]:
PIK_CIK = PIK.join(CIK, how='left')
print(PIK_CIK.to_latex(float_format='{:.4f}'.format))

In [ ]:


plt.figure(dpi=150)

plt.plot(obj.t_peak, obj.output, color='blue', label='True')
plt.plot(obj.t_peak, RF.predict(obj.iloc[:,:-1]), color='firebrick', label='Predicted')
plt.title('Test Predictions vs. True Output')
plt.ylabel('Risk Estimate')
plt.xlabel(r'$\tau$ (year of peak emissions)')
plt.legend()

## RF Per Year Category

In [20]:
u_years = obj['t_peak'].unique() # Obtain the unique years
mask = u_years<2100

In [15]:
time_PI = dict()
time_PIK = dict()
time_PT = dict()
for year in u_years[mask]:
    # if year>=2161:
    #     pass
    # else:
    #     # print(year)
    mask = obj['t_peak'] < year
    val1 = train_test_split(obj[mask].iloc[:,:-1], obj[mask].iloc[:,-1], train_size=0.5, random_state=0, shuffle=False)
    RF = RandomForestRegressor(n_jobs=10, random_state=0)
    RF.fit(val1[0], val1[2])
    time_PI[year] = mod_pvi1(val1[0], val1[2], RF.predict, n_boots=1000, alpha=0.05, column_set=npv_column_set)
    time_PT[year] = mod_pvi_t(val1[0], val1[2], RF.predict, n_boots=1000, alpha=0.05, column_set=npv_column_set)
    time_PIK[year] = mod_pvi2(val1[0], val1[2], RF.predict,S_I=time_PI[year], n_boots=1000, alpha=0.05, columns_set=npv_column_set)

In [19]:
res = pd.DataFrame(time_PT.values(), index=time_PT.keys())
res.to_excel(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Time_PVI\tot_time_PVI.xlsx")
res = pd.DataFrame(time_PI.values(), index=time_PI.keys())
res.to_excel(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Time_PVI\first_time_PVI.xlsx")


# Feature Selection

We will append a noise feature $S_{\mathrm{var}}\sim \mathcal{N}(\mathbf{\overline{0}}, \sigma^{2})$  to the dataset, to accentuate pertinent features. Thereafter, we select features based upon the total-order PVi contribution of $S_{\mathrm{var}}$, as all features contributing $\leqslant$ are equally insignificant.

In [6]:
# Redefing the TPVi for compatibility
def mpt(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,n_boots: int, alpha: float, column_set: list[str]) -> np.ndarray[float]:
    '''
    ===Total-Order Permutation Variable Importances===

        data: your training dataset
        y_pred: your model output (specifically the prediction of your 'data' arg)
        f: in this case, simply RF.predict
    '''
    rows, columns = data.shape
    npv_columns = column_set
    u = dict()
    V_y = np.var(y_true,ddof=1 )
    CI = dict()
    for col in npv_columns:
        d = data.copy()
        d[col] = np.roll(data[col],rows//2, axis=0)
        y_pred = f(d)
        u[col] = np.mean(np.square(y_true - y_pred))/(2*V_y)
        S_CI = bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
        p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
        CI[col] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'
    S_t = pd.DataFrame(u.values(), index=u.keys(), columns=[r'$P_{\tau}$'])
    return u, CI

In [7]:
new_obj = obj1.copy()
S_var = chi2.rvs(1,loc=0,scale=1, size=obj1.shape[0], random_state=0)
new_obj.insert(0, 'S_var',S_var)
# new_obj.sort_values(by='t_peak',ascending=True, inplace=True)

In [ ]:
new_obj.head(5)

In [ ]:
# Model Fitting
val1 = train_test_split(new_obj.iloc[:,:-1],new_obj.iloc[:,-1], train_size=0.8,random_state=0)
# val1 = TimeSeriesSplit(n_splits=4, max_train_size=60_066)
RF1 = RandomForestRegressor(n_estimators=300,n_jobs=20, random_state=0)
RF1.fit(val1[0],val1[2])

In [ ]:
RF1.feature_importances_

In [11]:
def mpv1(data: pd.DataFrame, y_true: np.ndarray[float], f: callable,*, n_boots: int, alpha: float, column_set: dict) -> tuple[pd.DataFrame]:
    rows, columns = data.shape
    npv_columns = column_set
    V_y = np.var(y_true, ddof=1)
    u = dict()
    CI = dict()
    rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    # rolled_data = np.roll(data.copy(), rows//2, axis=0)
    for col in npv_columns:
        d = rolled_data.copy()
        d[col] = data[col].to_numpy()
        y_pred = f(d)
        u[col] = 1-np.mean(np.square(y_true - y_pred))/(2*V_y)
        S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y)
        p0,p1 = np.quantile(S_CI, [alpha,1-alpha])
        CI[col] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'
    SI = pd.DataFrame(u.values(), index=u.keys(), columns=['$P_{i}$'])
    return u,CI

In [12]:
def mpv2(data: np.ndarray[float], y_true: np.ndarray[float], f: callable,*,S_I: np.ndarray[float] ,n_boots: int, alpha: float, columns_set: dict) -> tuple[np.ndarray[float], list]:
    '''
    ===Second-Order Permutation Variable Importances (Full Matrix)===

        data: your training dataset
        y_pred: your model output (specifically the y_true value)
        f: in this case, simply RF.predict

        S_I: First-Order Importances
        n_boots: number of bootstrap samples
        alpha: parameter for confidence interval [alpha, 1-alpha]
        S_CI: sampling distribution 
        CI: Confidence Intervals
    '''
    rows, columns = data.shape
    npv_columns = columns_set
    n = n_boots
    S = dict()
    CI = dict()
    V_y = np.var(y_true, ddof=1)
    rolled_data = pd.DataFrame(np.roll(data.copy(),rows//2, axis=0), columns=data.columns)
    # rolled_data = np.roll(data.copy(), rows//2, axis=0)
    for col1, col2 in it.combinations(npv_columns, r=2):
        d = rolled_data.copy()
        d[[col1,col2]] = data[[col1,col2]].to_numpy()
        y_pred = f(d)
        S_cat1 = S_I[col1]
        S_cat2 = S_I[col2]
        S[(col1, col2)] = 1 - np.mean(np.square(y_true - y_pred))/(2*V_y) - S_cat1 - S_cat2
        S_CI = 1 - bootstrapp(y_true,y_pred, n_boots)/(2*V_y) - S_cat1 - S_cat2 # Bootstrap Process
        p0,p1= np.quantile(S_CI, [alpha,1-alpha])
        CI[(col1, col2)] = [p0,p1,p1-p0]
    CI = pd.DataFrame(CI.values(), index = CI.keys(), columns=['5th','95th','Quantile Difference'])
    CI.index.name = 'Interactions'
    CI.columns.name = 'Confidence Interval Difference'

    # S = pd.DataFrame(S.values(), index=S.keys(), columns=[r'$P_{ik}$'])
    return S, CI

In [13]:
PI, CI = mpv1(val1[0], val1[2], RF1.predict, n_boots=1000, alpha=0.05, column_set=val1[0].columns)

In [ ]:
PI

In [17]:
PT, CT = mpt(val1[0], val1[2], RF1.predict, n_boots=1000, alpha=0.05, column_set=val1[0].columns)

In [ ]:
PT

In [ ]:
PT = pd.Series(PT.values(), index=PT.keys())
PT

In [15]:
PIK, CIK = mpv2(val1[0], val1[2], RF1.predict,S_I=PI, n_boots=1000, alpha=0.05, columns_set=val1[0].columns)

In [68]:
PIK = pd.DataFrame(PIK.values(), index=PIK.keys(), columns=[r'$P_{ik}$'])
PIK.to_excel(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\PVi_model_measures\Second_Order_PVI.xlsx")

In [79]:
# Features <= the importance of S_var
feat_tb_dropped = PT[PT<=PT.loc['S_var']].dropna()

In [80]:
feat_tb_dropped = set(feat_tb_dropped.index) - {'S_var'}

In [ ]:
len(list(feat_tb_dropped))

Now that I've found the features contributing little to the predicition of the model output, we can remove them and re-train, validate, and test our the new RF model.

**Reloading the dataset**

In [83]:
X = pd.read_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\ensemble_output\results\default\parameters.csv")
X.drop(columns=list(feat_tb_dropped), inplace=True)
Y = pd.read_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\ensemble_output\results\default\gmslr.csv")
Y = Y.mean(axis=1).rename('output')
obj = X.join(Y, how='left')
obj.sort_values(by='t_peak',ascending=True, inplace=True)
nun = len(X.t_peak.unique())
discret = KBinsDiscretizer(n_bins=nun, encode='ordinal', random_state=0)
Y_binned = discret.fit_transform(Y.to_numpy().reshape(-1,1)).ravel()
smote = SMOTENC(categorical_features=['t_peak'],sampling_strategy='auto', random_state=0)
X_res , y_res = smote.fit_resample(X, Y_binned)
Y1 = pd.Series(discret.inverse_transform(y_res.reshape(-1,1)).ravel(), name='output')
new_obj1 = X_res.join(Y1, how='left')
new_obj1.sort_values(by='t_peak',ascending=True, inplace=True)

In [ ]:
new_obj1

**Model Validation/Calibration No.1**\
As this is a time-series dataset, we define a new variable ```val2``` that partitions the dataset (w/o shuffling) into train/validation. When it's time to 'test' the performance of the model, we will switch to ```train_test_split```.

In [86]:
# val2 = train_test_split(obj.iloc[:,:-1],obj.iloc[:,-1],shuffle=False, train_size=0.8, random_state=0)
val2 = TimeSeriesSplit(n_splits=4, max_train_size=60_066)
RF = RandomForestRegressor(random_state=0)

**Model Validation/Calibration No.2**\
Defining the parameter space.

In [87]:
parameter_space = dict()
parameter_space['n_estimators'] = Integer(30,400)
# parameter_space['min_samples_split'] = Integer(2,5, prior='uniform')
# parameter_space['min_samples_leaf'] = Integer(1,5)
parameter_space['max_features'] = Integer(5,22)
parameter_space['max_depth'] = Integer(10,80)
# parameter_space['ccp_alpha'] = Real(1e-4, 5e-4)

In [ ]:
bae = BayesSearchCV(estimator=RF,search_spaces=parameter_space,verbose=2, n_iter=100, scoring='neg_mean_squared_error', n_points=6, cv=val2, n_jobs=24, random_state=0)
bae.fit(new_obj1.iloc[:,:-1], new_obj1.iloc[:,-1])
res = pd.DataFrame(bae.cv_results_)
res.to_excel(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Hyperparameter_Tuning\Darnell_HT_MSE_FS_3.xlsx")
res.to_csv(r"C:\Users\dwigh\OneDrive\Desktop\Emissions Uncertainty on Antarctic Instability\Hyperparameter_Tuning\Darnell_HT_MSE_FS_3.csv")
print(f'Best Parameters:{bae.best_params_} with MSE: {bae.best_score_}')

In [ ]:
val2 = train_test_split(new_obj1.iloc[:,:-1],new_obj1.iloc[:,-1],shuffle=False, train_size=0.8, random_state=0)
RF1 = RandomForestRegressor(n_estimators=400, max_features=18, max_depth=76, n_jobs=20, random_state=0)
RF1.fit(val2[0], val2[2])

In [ ]:
val2[0].columns

In [102]:
Variables = ['sd_temp', 'sd_ocean_heat', 'sd_glaciers', 'sd_greenland','sd_antarctic',
              'sd_gmsl', 'rho_temperature','rho_ocean_heat', 
              'rho_glaciers', 'rho_greenland', 'rho_antarctic','rho_gmsl']
Climate = ['CO2_0', 'N2O_0', 'temperature_0','ocean_heat_0','Q10', 'CO2_fertilization',
           'CO2_diffusivity', 'heat_diffusivity', 'rf_scale_aerosol','climate_sensitivity']
Glaciers = ['glaciers_v0','glaciers_s0','glaciers_beta0', 'glaciers_n']
Thermals = ['thermal_s0', 'thermal_alpha']

Greenland = ['greenland_a', 'greenland_b','greenland_alpha', 'greenland_beta']
Antarctic = [ 'antarctic_gamma', 'antarctic_alpha','antarctic_mu', 'antarctic_nu',
              'antarctic_precip0', 'antarctic_kappa','antarctic_flow0', 'antarctic_runoff_height0',
                'antarctic_c','antarctic_bed_height0', 'antarctic_slope', 'antarctic_lambda','antarctic_temp_threshold',
                'antarctic_s0','anto_alpha', 'anto_beta']
variables = list(set(Variables) & set(val2[0].columns))
climate = list(set(Climate) & set(val2[0].columns))
thermals = list(set(Thermals) & set(val2[0].columns))
greenland = list(set(Greenland) & set(val2[0].columns))
antarctic = list(set(Antarctic) & set(val2[0].columns))
npv_column_set = dict()
npv_column_set['Variables'] = variables
npv_column_set['Climate'] = climate
# npv_column_set['Glaciers'] = Glaciers
npv_column_set['Thermals'] = thermals
npv_column_set['Greenland'] = greenland
npv_column_set['Antarctic'] = antarctic
npv_column_set['Emissions'] = ['gamma_g', 't_peak', 'gamma_d', 'lw_random_sample']

In [ ]:
print(f'Train MSE={np.mean(np.square(val2[2].to_numpy()-RF1.predict(val2[0].to_numpy())))}')
print(f'Test MSE={np.mean(np.square(val2[3].to_numpy()-RF1.predict(val2[1].to_numpy())))}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi=150)

plt.plot(val2[1].t_peak, val2[3], color='blue', label='True')
plt.plot(val2[1].t_peak, RF1.predict(val2[1]), color='firebrick', label='Predicted')
plt.title('Test Predictions vs. True Output')
plt.legend()

In [103]:
PI, CI = mod_pvi1(val2[0], val2[2], RF1.predict, n_boots=1000, alpha=0.05, column_set=npv_column_set)
PIK, CIK = mod_pvi2(val2[0], val2[2], RF1.predict,S_I=PI, n_boots=1000, alpha=0.05, columns_set=npv_column_set)

In [ ]:
PI = pd.DataFrame(PI.values(), index=PI.keys(), columns=[r'$P_{i}$'])
PIK = pd.DataFrame(PIK.values(), index=PIK.keys(), columns=[r'$P_{ik}$'])

In [109]:
PI_confide = PI.join(CI, how='left')
PIK_confide = PIK.join(CIK, how='left')

In [ ]:
print(PI_confide.to_latex(float_format='{:.4f}'.format))

In [ ]:
print(PIK_confide.to_latex(float_format='{:.4f}'.format))

In [119]:
# PI[PI>0].dropna()['$P_{i}$'].to_dict()
pik = PIK.to_dict()['$P_{ik}$']
ref_pik = {','.join(key): pik[key] for key in pik.keys()}
merged_measures =  PI[PI>0].dropna()['$P_{i}$'].to_dict()|ref_pik

In [120]:
values = list(merged_measures.values())
labels = list(merged_measures.keys())

In [121]:
PT, CT = mod_pvi_t(val2[0], val2[2], RF1.predict, n_boots=1000, alpha=0.05, column_set=npv_column_set)

In [124]:
values2 = list(PT.values())
labels2 = list(PT.keys())

In [139]:
def donut_chart(values: np.ndarray, values1: np.ndarray, labels: list,labels1: list, props: dict, kw: dict):
    fig, ax= plt.subplots(1,2, dpi=400, figsize=(20,25))
    wedges0, text0 = ax[0].pie(values,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
    for i, p in enumerate(wedges0):
        ang = (p.theta2-p.theta1)/2 + p.theta1
        x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
        horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
        connectionstyle = f'angle, angleA=0, angleB={ang}'
        kw['arrowprops'].update({'connectionstyle':connectionstyle})
        ax[0].annotate(f'{values[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
    ax[0].set_title('SNEASY-BRICK: First & Second-Order PVi')
    ax[0].legend(labels=labels,loc='center', title='PVi Indices:' )

    wedges1, text1 = ax[1].pie(values1,wedgeprops=props['wedgeprops'],colors=props['colors'], startangle= props['startangle'], textprops = props['textprops'])
    for i, p in enumerate(wedges1):
        ang = (p.theta2-p.theta1)/2 + p.theta1
        x,y = np.cos(np.deg2rad(ang)), np.sin(np.deg2rad(ang))
        horizontalalignment = {-1:'right', 1:'left'}[int(np.sign(x))]
        connectionstyle = f'angle, angleA=0, angleB={ang}'
        kw['arrowprops'].update({'connectionstyle':connectionstyle})
        ax[1].annotate(f'{values1[i]: .1%}', xy=(x,y), xytext=(1.1*np.sign(x),1.2*y), horizontalalignment=horizontalalignment, **kw)
    ax[1].set_title('SNEASY-BRICK: Total-Order PVi')
    ax[1].legend(labels=labels1,loc='center', title='PVi Indices:' )
    plt.show()

In [ ]:
donut_chart(values, values2, labels, labels2, props, kw)